<a href="https://colab.research.google.com/github/Quanhcmus/Image_classification_CIFAR10_with_VGG11/blob/main/Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from sklearn.model_selection import train_test_split


num_class=10
batch_size=32
epoch=50
data_augmentation = True

(X,y),(X_test,y_test)=cifar10.load_data()
X_train,X_valid,y_train,y_valid=train_test_split(X,y,test_size=0.25,random_state=42)
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_valid.shape[0], 'valid samples')
print(X_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
# Convert class vectors to binary class matrices.
y_train=keras.utils.to_categorical(y_train)
y_test=keras.utils.to_categorical(y_test)

In [ ]:
# tất cả các pixel của ảnh đều nằm trong khoảnh [0;255] nên chia cho 255 để tất cả đều nằm trong khoảng [0;1]
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
X_train/=255
X_test/=255

In [ ]:
# VGG model
model=Sequential()

# block 1
model.add(Conv2D(64,(3,3),padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2,strides=2))
# block 2
model.add(Conv2D(128,(3,3),padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2,strides=2))
# block 3
model.add(Conv2D(256,(3,3),padding='same',input_shape=X_train.shape[1:]))
model.add(Conv2D(256,(3,3),padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2,strides=2))
# block 4
model.add(Conv2D(512,(3,3),padding='same',input_shape=X_train.shape[1:]))
model.add(Conv2D(512,(3,3),padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2,strides=2))
# block 5
model.add(Conv2D(512,(3,3),padding='same',input_shape=X_train.shape[1:]))
model.add(Conv2D(512,(3,3),padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(GlobalMaxPooling2D())
# FL 4096x2 Fully conected layer
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.25))
# FL 10
model.add(Dense(10))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.Adam(lr=0.0001)
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 activation_1 (Activation)   (None, 16, 16, 128)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                    

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph2', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epoch,
              validation_data=(X_test, y_test),
              shuffle=True, callbacks=[tbCallBack])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    '''
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    '''
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=epoch,
                        validation_data=(X_test, y_test), callbacks=[tbCallBack])


Using real-time data augmentation.
Epoch 1/50


<ipython-input-6-fafaa43913b4>:42: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(X_train, y_train,


1562/1562 [==============================] - 49s 29ms/step - loss: 1.6843 - accuracy: 0.3585 - val_loss: 1.2790 - val_accuracy: 0.5254
Epoch 2/50
1562/1562 [==============================] - 43s 28ms/step - loss: 1.2404 - accuracy: 0.5493 - val_loss: 0.9976 - val_accuracy: 0.6343
Epoch 3/50
1562/1562 [==============================] - 44s 28ms/step - loss: 1.0317 - accuracy: 0.6317 - val_loss: 0.8689 - val_accuracy: 0.6942
Epoch 4/50
1562/1562 [==============================] - 44s 28ms/step - loss: 0.9120 - accuracy: 0.6794 - val_loss: 0.8214 - val_accuracy: 0.7170
Epoch 5/50
1562/1562 [==============================] - 44s 28ms/step - loss: 0.8290 - accuracy: 0.7117 - val_loss: 0.7852 - val_accuracy: 0.7318
Epoch 6/50
1562/1562 [==============================] - 43s 28ms/step - loss: 0.7713 - accuracy: 0.7316 - val_loss: 0.7208 - val_accuracy: 0.7504
Epoch 7/50
1562/1562 [==============================] - 45s 29ms/step - loss: 0.7175 - accuracy: 0.7502 - val_loss: 0.7024 - val_accura

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
scores=model.evaluate(X_test,y_test,verbose=1)
print("Test Loss",scores[0])
print("Test Accuracy",scores[1])

313/313 [==============================] - 2s 6ms/step - loss: 0.5051 - accuracy: 0.8482
Test Loss 0.5051476359367371
Test Accuracy 0.8482000231742859
